## ATU : PFDA Webservice And Applications

__Assignment 2 : Card Shark__

__Student: Clyde Watts__

__ Date : 2026-01-30 __

__Opened:__ Saturday, 3 January 2026, 11:14 AM
__Due:__ Monday, 20 April 2026, 12:14 PM
### Assignment
look at the the page

Deck of Cards API    https://deckofcardsapi.com/

This is an API that simulates dealing a deck of cards

Write a program that "deals" (prints out) 5 cards

first you need to shuffle

https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1

get the deck_id, 

with the deck_id you can get the cards

https://deckofcardsapi.com/api/deck/<<deck_id>>/draw/?count=2

This example gets two cards

From there you need to print the value and the suit of each card.

save the file as assignment2-carddraw.py (or as a notebook)

Last few marks:
Check if the user has drawn a pair, triple, straight, or all of the same suit and congratulate the user.


### Load Libraries

In [1]:
import requests
import csv
from xml.dom.minidom import parseString
# used to export json data nicely
import pprint
import os
from IPython.display import Image, display
import pandas as pd

## Create a class to simplify handling the dec

In [2]:
deckofcardsURL = "https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1"
card_value_map={}
card_value_map['2'] = 2
card_value_map['3'] = 3
card_value_map['4'] = 4
card_value_map['5'] = 5
card_value_map['6'] = 6
card_value_map['7'] = 7
card_value_map['8'] = 8
card_value_map['9'] = 9
card_value_map['0'] = 10
card_value_map['J'] = 11
card_value_map['Q'] = 12
card_value_map['K'] = 13
card_value_map['A'] = 14

def deck_new(deck_count:str=1):
    """Create a new shuffled deck of cards. 
    Args:
        deck_count (str, optional): Number of decks to include in the shuffle. Defaults to 1. """
    url = f"https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count={deck_count}"
    response = requests.get(url)
    return response.json()

def deck_draw(deck_id:str, count:str=1):
    """Draw cards from a deck.
    Args:
        deck_id (str): The ID of the deck to draw from.
        count (str, optional): Number of cards to draw. Defaults to 1. """
    url = f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count={count}"
    response = requests.get(url)
    return response.json()

def evaluate_hand(pocker_hand):
    """
    Evaluate a poker hand and return its type and score.
    Args:
        hand (list): A list of strings representing the cards in the hand (e.g., ['10H', 'JH', 'QH', 'KH', 'AH']).
        Returns:    
        tuple: A tuple containing the type of hand (str), the score of the hand (int), and a tie breaker score (int).
    """
    hand_unsorted = []
    image_filenames = []
    for card in pocker_hand['cards']:
        # get the image URL
        card_image_url = card['image']
        # image filename
        image_filename = f"{card['code']}.png"
        hand_unsorted.append(card['code'])
        # get the image and save it locally
        # check if file already exists
        if not os.path.exists(image_filename):
            image_response = requests.get(card_image_url)
            # save the image to a file
            with open(image_filename, 'wb') as image_file:
                image_file.write(image_response.content)
        image_filenames.append(image_filename)
    hand = hand_unsorted.copy()
    hand.sort()
    # normalize parse: value is everything except last char, suit is last char
    values = []
    value_ints = []
    suits = []
    for c in hand:
        v = c[:-1]
        s = c[-1]
        # normalize occasional full names
        if v.upper() == 'ACE':
            v = 'A'
        if v == '10' and '10' not in card_value_map:
            # ensure map handles '10'
            card_value_map['10'] = 10
        # prefer mapping by exact token if available, otherwise try first char or int conversion
        if v in card_value_map:
            vi = card_value_map[v]
        else:
            try:
                vi = int(v)
            except Exception:
                vi = card_value_map.get(v[0], 0)
        values.append(v)
        value_ints.append(vi)
        suits.append(s)

    df = pd.DataFrame({
        'Suit' : [i[-1] for i in hand],
        'Value': [i[:-1] for i in hand],
        'ValueInt': [card_value_map[i[:-1]] for i in hand],
        'Hand': hand
    })

    suit_counts = df.groupby('Suit').size()
    #print(f"Suite Counts: {suit_counts}")
    max_suit_counts = suit_counts.max()
    #print(f"Max suit counts: {max_suit_counts}")
    value_counts = df.groupby('Value').size()
    #print("Value of hand:")
    #print(f"Value counts: {value_counts}")
    max_value_counts = value_counts.max()
    #print(f"Max value counts: {max_value_counts}")
    total_values = df['ValueInt'].sum()
    #print("Total value of hand:")
    #print(total_values)
    highest_value = df['ValueInt'].max()
    #print("Highest value in hand:")
    #print(highest_value)
    lowest_value = df['ValueInt'].min()
    #print("Lowest value in hand:")
    #print(lowest_value)
    hi_lo_difference = highest_value - lowest_value
    #print("Difference between highest and lowest value:")
    #print(hi_lo_difference)
    count_value_counts = value_counts.value_counts().sort_values(ascending=False)
    #print("Count of distinct card values in hand:")
    #print(count_value_counts)
    type_of_hand = "Unknown"
    type_of_hand_score = 0
    tie_breaker_score = 0 # TODO : implement tie breaker score
    if max_suit_counts == 5 and hi_lo_difference == 4 and highest_value == 14: # all cards same suit, in sequence, and highest is Ace
        type_of_hand = "Royal Flush"
        type_of_hand_score = 10
    elif max_suit_counts == 5 and hi_lo_difference == 4: # all cards same suit and in sequence
        type_of_hand = "Straight Flush"
        type_of_hand_score = 9
    elif max_value_counts == 4: # four of a kind
        type_of_hand = "Four of a Kind"
        type_of_hand_score = 8
    elif max_suit_counts == 5:
        type_of_hand = "Flush"
        type_of_hand_score = 7
    elif hi_lo_difference == 4 and max_value_counts == 1:
        type_of_hand = "Straight"
        type_of_hand_score = 6
    elif max_value_counts == 3:
        type_of_hand = "Three of a Kind"
        type_of_hand_score = 5
    elif max_value_counts == 3 and count_value_counts.get(2,0) == 1:
        type_of_hand = "Full House"
        type_of_hand_score = 4
    elif max_value_counts == 2 and count_value_counts.get(2,0) == 2:
        type_of_hand = "Two Pair"
        type_of_hand_score = 3
    elif max_value_counts == 2:
        type_of_hand = "One Pair"
        type_of_hand_score = 2
    else:
        type_of_hand = "High Card"
        type_of_hand_score = 1
    
    #print(f"Type of hand: {type_of_hand} (Score: {type_of_hand_score})")
    return type_of_hand, type_of_hand_score, tie_breaker_score, hand_unsorted, image_filenames



## Get 5 Hands

Get 5 hands and display raw json

In [3]:
deck = deck_new()
deck_id = deck['deck_id']
pocker_hand = deck_draw(deck_id, count=5)
pprint.pprint(pocker_hand)

{'cards': [{'code': 'JH',
            'image': 'https://deckofcardsapi.com/static/img/JH.png',
            'images': {'png': 'https://deckofcardsapi.com/static/img/JH.png',
                       'svg': 'https://deckofcardsapi.com/static/img/JH.svg'},
            'suit': 'HEARTS',
            'value': 'JACK'},
           {'code': '6D',
            'image': 'https://deckofcardsapi.com/static/img/6D.png',
            'images': {'png': 'https://deckofcardsapi.com/static/img/6D.png',
                       'svg': 'https://deckofcardsapi.com/static/img/6D.svg'},
            'suit': 'DIAMONDS',
            'value': '6'},
           {'code': 'QD',
            'image': 'https://deckofcardsapi.com/static/img/QD.png',
            'images': {'png': 'https://deckofcardsapi.com/static/img/QD.png',
                       'svg': 'https://deckofcardsapi.com/static/img/QD.svg'},
            'suit': 'DIAMONDS',
            'value': 'QUEEN'},
           {'code': '3H',
            'image': 'https://deckof

#### Task 1

From there you need to print the value and the suit of each card.

In [4]:
for card in pocker_hand['cards']:
    print(f"{card['value']} of {card['suit']}")

JACK of HEARTS
6 of DIAMONDS
QUEEN of DIAMONDS
3 of HEARTS
4 of HEARTS


__That is boring__

Print images of cards 

In [5]:
# Coding Agent Prompt : display cards horizontally
import code


image_filenames = []
# Loop through each card and download its image
hand = []
for card in pocker_hand['cards']:
    # get the image URL
    card_image_url = card['image']
    # image filename
    image_filename = f"{card['code']}.png"
    hand.append(card['code'])
    # get the image and save it locally
    # check if file already exists
    if not os.path.exists(image_filename):
        image_response = requests.get(card_image_url)
        # save the image to a file
        with open(image_filename, 'wb') as image_file:
            image_file.write(image_response.content)
    image_filenames.append(image_filename)

# display the images horizontally
from IPython.display import HTML
imgs_html = ''.join(f'<img src="{fn}" style="height:200px; margin-right:10px;">' for fn in image_filenames)
display(HTML(f'<div style="display:flex; align-items:flex-start;">{imgs_html}</div>'))


### Task 2 - Extra Credits - Score Hand


Check if the user has drawn a pair, triple, straight, or all of the same suit and congratulate the user.


__How__

There are a number of approaches , I will use panda dataframe.
This will classify the hand 

__royal flush__

max_suit_counts == 5 and highest_value = 14 and hi_lo_difference = 14-10 = 4

__staight flush__

max_suit_counts == 5 and  hi_lo_difference = 14-10 = 4

__four of kind__

max_suit_count == 4

__flush__ 

max_suit_counts == 5

__straight__

hi_lo_difference=4 and max_value_counts == 1

__three_of_kind__

max_suit_counts == 5


__two_pair__

max_value_count == 2 and count_value_counts[<get pairs>] = 2

__one_pair__

max_value_count == 2 and count_value_counts[<get pairs>] = 1

__high_chard__







In [9]:
deck = deck_new()
deck_id = deck['deck_id']
pointsA = 0
pointsB = 0
hand = deck_draw(deck_id, count=5)
#def evaluate_hand
type_of_hand, type_of_hand_score, tie_breaker_score,hand_sorted,image_filenames = evaluate_hand(hand)
print(f"Hand drawn: {hand_sorted}")
print(f"Final Evaluation: {type_of_hand} (Score: {type_of_hand_score}, Tie Breaker Score: {tie_breaker_score})")
print("Well Done!")




Hand drawn: ['7D', '9D', 'QH', 'KD', '3D']
Final Evaluation: High Card (Score: 1, Tie Breaker Score: 0)
Well Done!


## A short card game

A card game is played with one deck of cards , there are 2 plays drawing 5 cards each , and then comparing there hands
When the deck is empty , after 5 games . The winner is declaired. 

A game can be won , lost or draw.

In [7]:
# Draw a deck of cards
deck = deck_new()
deck_id = deck['deck_id']
pointsA = 0
pointsB = 0
for _i in range(5):
    hand_A = deck_draw(deck_id, count=5)
    hand_B = deck_draw(deck_id, count=5)

    print(f"Player A's Hand:")
    type_of_hand_A, type_of_hand_score_A, tie_breaker_score_A, handA_unsorted, handA_images = evaluate_hand(hand_A)
    print(f"Final Evaluation for Player A: {type_of_hand_A} (Score: {type_of_hand_score_A}, Tie Breaker Score: {tie_breaker_score_A})")
    print(f"Player B's Hand:")
    type_of_hand_B, type_of_hand_score_B, tie_breaker_score_B, handB_unsorted, handB_images = evaluate_hand(hand_B)
    print(f"Final Evaluation for Player B: {type_of_hand_B} (Score: {type_of_hand_score_B}, Tie Breaker Score: {tie_breaker_score_B})")
    if type_of_hand_score_A > type_of_hand_score_B:
        pointsA += 1
        print("Player A wins this round!")
    elif type_of_hand_score_B > type_of_hand_score_A:
        pointsB += 1
        print("Player B wins this round!")
    else:
        print("This round is a tie!")
print(f"Final Score: Player A: {pointsA}, Player B: {pointsB}")
    

Player A's Hand:
Final Evaluation for Player A: High Card (Score: 1, Tie Breaker Score: 0)
Player B's Hand:
Final Evaluation for Player B: One Pair (Score: 2, Tie Breaker Score: 0)
Player B wins this round!
Player A's Hand:
Final Evaluation for Player A: One Pair (Score: 2, Tie Breaker Score: 0)
Player B's Hand:
Final Evaluation for Player B: High Card (Score: 1, Tie Breaker Score: 0)
Player A wins this round!
Player A's Hand:
Final Evaluation for Player A: High Card (Score: 1, Tie Breaker Score: 0)
Player B's Hand:
Final Evaluation for Player B: One Pair (Score: 2, Tie Breaker Score: 0)
Player B wins this round!
Player A's Hand:
Final Evaluation for Player A: High Card (Score: 1, Tie Breaker Score: 0)
Player B's Hand:
Final Evaluation for Player B: High Card (Score: 1, Tie Breaker Score: 0)
This round is a tie!
Player A's Hand:
Final Evaluation for Player A: High Card (Score: 1, Tie Breaker Score: 0)
Player B's Hand:
Final Evaluation for Player B: High Card (Score: 1, Tie Breaker Scor